In [8]:
import pandas as pd
import numpy as np
import transformers
from collections import Counter

In [9]:
dataset = pd.read_csv('Mis7B_Mistral_bills5000_for_dpo.csv')

In [10]:
# dd = pd.read_csv('testset_with_topics.csv')
data = pd.DataFrame({'text': dataset.prompt, 'raw_topics': dataset.outputs})

In [11]:
data.head(2)

,text,raw_topics
0,"<s>[INST] Based on the provided text, identify...",1. Atomic Veterans\n2. Healthcare\n3. Parity Act
1,"<s>[INST] Based on the provided text, identify...",1. Appalachian Region\n2. Development Efforts\...


In [12]:
from typing import Dict, Optional
def return_prompt_and_responses(content, accept, reject) -> Dict[str, str]:
        return {"prompt": content,"chosen": accept,"rejected": reject}

In [13]:
def shorten_string_before_marker(input_string, marker):
    # Find the position of the marker
    marker_position = input_string.find(marker)
    # If the marker is found, return the substring up to the marker
    if marker_position != -1:
        return input_string[marker_position:]
    else:
        # If the marker is not found, return the original string
        return input_string
# sample_string = "This is a sample string. Topic 1: Some topic content."
# shortened_string = shorten_string_before_marker(sample_string)

In [14]:
def number_topics(input_string):
    # Split the string by newline character
    topics = input_string.split('\n')
    
    # Enumerate and format each topic with its corresponding number
    numbered_topics = [f'Topic {i + 1}: {topic.split(" : ")[1]}' for i, topic in enumerate(topics)]
    
    # Join the formatted strings back together
    return '\n'.join(numbered_topics)

# Example usage
# input_string = 'Topic : Attitude\nTopic : Immunity\nTopic : Among'
# output_string = number_topics(input_string)
# print(output_string)


In [15]:
import re
#select first three sentence
def take_first_three(text):
# Use regular expression to match lines starting with 1., 2., or 3.
    topics = re.findall(r'^\d+\.\s.*$', text, re.M)
    first_three_topics = topics[:3]  # Get the first three topics
    a = "\n".join(first_three_topics)
    return a
# print("\n".join(first_three_topics))


In [19]:
# check out if any topics in the "match and replace list"
def check_string_in_dataframe(df, string_to_check):
    for column in df.columns:
        # Check if any cell in the column contains the exact string
        if df[column].apply(lambda x: string_to_check in x if isinstance(x, list) else string_to_check == x).any():
            return column
    return string_to_check

In [16]:
clean = [i.replace("\\'", "'") for i in data.text]
data['text'] = clean
take_3lines = [take_first_three(i) for i in data.raw_topics]
data['raw_topics'] = take_3lines

In [17]:
# match and replace list:
grouped_topics_df = pd.read_csv('./data/match_replace_for_bill.csv')

In [20]:
RLHFx = []
for i in range(len(data)):
    #content = prompt_sample_before_no_format.format(list_of_text = data['text'][i])
    content = data['text'][i]
    #
    ss = data.raw_topics[i]
    #ss = shorten_string_before_marker(ss, marker = '1. ')
    if ss[:1] == '\n':
        ss = ss.replace('\n','',1).replace('\n\n','\n')
    else:
        ss = ss.replace('\n\n','\n')
#     if ss[-1] == '\n':
#         ss = ss[:len(ss)-1]
    ss = ss.replace('1. ','').replace('\n2. ', '$$').replace('\n3. ','$$')
#     ss = ss.lower()
    sss = ss.split('$$')
    #
    if len(sss) == 1:
        s1 = check_string_in_dataframe(grouped_topics_df, sss[0])
    if len(sss) == 2:
        s1 = check_string_in_dataframe(grouped_topics_df, sss[0])
        s2 = check_string_in_dataframe(grouped_topics_df, sss[1])
    if len(sss) == 3:
        s1 = check_string_in_dataframe(grouped_topics_df, sss[0])
        s2 = check_string_in_dataframe(grouped_topics_df, sss[1])
        s3 = check_string_in_dataframe(grouped_topics_df, sss[2])
    #
    accept = f'''1. {s1}\n2. {s2}\n3. {s3}'''
    if accept[-1] == '\n':
        accept = accept[:len(accept)-1]
    #accept = replace_items(accept, items_to_replace)
#     accept = shorten_string_before_marker(accept, marker = 'Topic 1:')
    #accept = number_topics(accept)
    xx = data.raw_topics[i]#.replace('1','').replace('2','').replace('3','')
    
    if xx[:1] == '\n':
        xx = xx.replace('\n','',1)
    else:
        xx = xx.replace('\n\n','\n')
    rejected = f'''{xx}'''
    #if rejected[-1] == '\n':
     #   rejected = rejected[:len(rejected)-1]
    if accept != rejected:
        print('yes')
        text = return_prompt_and_responses(content, accept, rejected)
        RLHFx.append(text)
    else:
        print('no')
#         text = return_prompt_and_responses(content, accept, rejected)
#         RLHF.append(text)
#     RLHF.append(text)

yes
no
no
yes
no
yes
yes
no
no
yes
yes
yes
yes
yes
no
no
yes
yes
yes
no
no
no
yes
no
no
yes
yes
yes
no
yes
no
no
yes
no
no
no
yes
yes
no
yes
no
no
yes
no
no
no
no
yes
no
yes
no
no
yes
yes
yes
yes
no
yes
yes
no
no
yes
no
no
no
no
yes
yes
no
yes
yes
no
yes
yes
yes
yes
yes
no
no
no
no
no
no
no
yes
yes
no
yes
no
no
yes
no
no
yes
yes
no
no
no
no
no
yes
no
no
no
yes
yes
no
no
no
yes
no
yes
yes
yes
yes
no
yes
yes
no
no
no
no
no
no
yes
no
no
no
yes
no
no
no
no
no
no
no
yes
yes
yes
yes
yes
yes
no
no
no
no
yes
yes
yes
yes
yes
yes
no
no
yes
no
no
no
yes
yes
yes
no
yes
yes
no
no
no
no
no
yes
yes
no
no
yes
no
no
yes
no
no
no
no
no
yes
no
yes
yes
yes
no
no
no
yes
yes
yes
no
yes
yes
yes
yes
no
yes
yes
no
yes
no
yes
no
no
no
no
yes
yes
no
no
yes
no
yes
yes
no
yes
no
no
yes
no
yes
yes
yes
no
yes
no
yes
no
yes
yes
yes
no
yes
yes
no
no
yes
no
yes
yes
no
no
no
yes
yes
no
no
no
no
no
no
no
yes
no
no
yes
yes
yes
no
no
yes
no
no
no
no
yes
no
no
no
no
yes
yes
yes
no
no
yes
yes
no
yes
yes
no
yes
no
no
no
yes
y

no
no
yes
no
no
yes
no
yes
no
no
yes
yes
no
yes
no
yes
yes
no
yes
yes
no
no
yes
no
no
yes
yes
no
no
yes
yes
no
no
yes
yes
no
no
yes
yes
no
no
yes
no
no
no
yes
yes
no
yes
no
no
yes
yes
no
yes
yes
yes
yes
no
yes
yes
yes
no
no
yes
yes
yes
yes
no
yes
no
no
no
no
yes
no
no
yes
yes
no
no
no
no
yes
no
no
yes
no
no
yes
no
yes
no
yes
no
no
no
no
yes
yes
no
no
no
yes
no
yes
yes
yes
no
yes
no
no
yes
no
yes
no
yes
yes
yes
no
no
yes
yes
no
yes
no
no
no
yes
no
no
yes
yes
no
yes
yes
no
no
yes
no
no
no
yes
yes
yes
no
no
no
yes
no
no
no
no
yes
yes
no
no
no
yes
no
yes
yes
no
yes
no
yes
yes
no
no
no
no
no
yes
yes
no
yes
no
no
yes
no
no
no
no
yes
no
no
yes
yes
no
no
no
no
yes
no
yes
no
no
no
yes
yes
no
no
yes
yes
yes
yes
no
no
no
no
no
no
yes
no
yes
no
no
no
yes
no
no
yes
no
no
no
no
no
no
no
no
yes
no
yes
no
no
no
yes
yes
yes
no
no
yes
yes
no
yes
no
yes
no
yes
no
no
no
yes
yes
no
yes
no
no
no
no
no
no
no
yes
yes
no
no
no
yes
no
yes
yes
no
no
no
no
yes
no
yes
yes
no
no
no
yes
no
yes
no
yes
no
no
no
no
yes

no
no
no
yes
yes
no
no
no
no
yes
yes
no
no
no
no
no
no
no
no
yes
no
yes
yes
no
yes
no
yes
no
no
no
no
no
no
yes
yes
no
yes
no
yes
no
no
no
no
no
no
no
yes
no
yes
yes
yes
no
no
no
yes
yes
yes
yes
yes
yes
yes
no
no
no
no
yes
yes
no
yes
no
no
yes
yes
no
no
yes
yes
yes
no
no
yes
no
yes
yes
no
no
no
yes
yes
no
yes
no
yes
no
no
no
no
no
yes
yes
yes
no
yes
yes
no
no
yes
yes
yes
yes
no
no
no
no
yes
yes
no
no
no
no
yes
yes
yes
no
no
no
yes
yes
yes
yes
no
no
no
no
no
yes
no
yes
yes
no
yes
yes
yes
no
no
no
no
no
no
yes
no
yes
no
yes
yes
no
yes
yes
yes
yes
yes
yes
yes
no
yes
no
no
no
yes
no
no
no
yes
yes
no
no
no
yes
no
no
no
yes
no
no
yes
yes
yes
yes


In [21]:
RLHF= pd.DataFrame(RLHFx)
RLHF.columns=['question', 'response_j', 'response_k']
# RLHF.to_csv('RLHF_naming_30Jan.csv', index = False)

In [22]:
RLHF

,question,response_j,response_k
0,"<s>[INST] Based on the provided text, identify...",1. Veterans\n2. Healthcare\n3. Parity Act,1. Atomic Veterans\n2. Healthcare\n3. Parity Act
1,"<s>[INST] Based on the provided text, identify...",1. Military\n2. Housing\n3. Distance Learning,1. Veterans\n2. Housing Assistance\n3. Distanc...
2,"<s>[INST] Based on the provided text, identify...",1. Veterans\n2. Benefit Payment\n3. Training H...,1. Veterans Education\n2. Benefit Payment\n3. ...
3,"<s>[INST] Based on the provided text, identify...",1. Military\n2. Nutrition\n3. Fairness,1. Veterans\n2. Nutrition\n3. Fairness
4,"<s>[INST] Based on the provided text, identify...",1. Veterans\n2. Outreach Services\n3. Hospital...,1. Veterans' Education\n2. Outreach Services\n...
...,...,...,...
2152,"<s>[INST] Based on the provided text, identify...",1. Tax\n2. Building Rehabilitation\n3. Energy,1. Tax Credits\n2. Building Rehabilitation\n3....
2153,"<s>[INST] Based on the provided text, identify...",1. Healthcare\n2. Congress\n3. FEHB Program,1. Health Insurance\n2. Congress\n3. FEHB Program
2154,"<s>[INST] Based on the provided text, identify...",1. Counter-Terrorism\n2. National Security\n3....,1. Counter-Terrorism\n2. National Security\n3....
2155,"<s>[INST] Based on the provided text, identify...",1. Video Games\n2. Ratings\n3. Law,1. Video Games\n2. Ratings\n3. Enforcement


In [ ]:
#RLHF.to_csv('bills_all_samples_for_dpo.csv', index=False)